In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import mglearn
from IPython.display import display, clear_output
import shap
from sklearn.tree import plot_tree

In [ ]:
# --- Data Loading and Preprocessing ---
# User configuration
base_path = 'your_data_directory_here/' # Set your preferred target data directory
file_name = 'Radical_Analysis.xlsx'

gbdt_path = os.path.join(base_path, 'GBDT')
os.makedirs(gbdt_path, exist_ok=True)

file_path = os.path.join(base_path, file_name)
df = pd.read_excel(file_path)
df.head()

X = df[['NH2', 'NH', 'N-(C)3', 'OH', 'CHO', 'COOH', 'Size']]
y = df['r']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)

In [ ]:
# --- Hyperparameter Tuning via Grid Search ---
gbr = GradientBoostingRegressor(random_state=42)
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

param_grid = {
    'n_estimators': range(10, 101, 5),
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': range(2, 21),
    'subsample': [0.7, 0.8, 1.0],
    'min_samples_split': range(3, 12, 2)
}

grid_search = GridSearchCV(estimator=gbr, param_grid=param_grid, cv=kfold, n_jobs=-1, scoring='neg_mean_squared_error', verbose=3)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_

clear_output(wait=True)
display("Grid search complete!")

print(f'Best Parameters: {grid_search.best_params_}')
print(f'Best scores: {grid_search.best_score_}')

In [ ]:
# --- Parameter Sensitivity Analysis (n_estimators vs max_depth) ---
n_estimators_range = range(10, 101, 5)
max_depth_range = range(1, 21)

CVS_matrix = np.zeros((len(n_estimators_range), len(max_depth_range)))
rmse_matrix = np.zeros((len(n_estimators_range), len(max_depth_range)))
R2_matrix = np.zeros((len(n_estimators_range), len(max_depth_range)))

for i, n_estimators in enumerate(n_estimators_range):
    for j, max_depth in enumerate(max_depth_range):
        # Using best parameters from GridSearch for fixed variables
        gbr = GradientBoostingRegressor(n_estimators=n_estimators, 
                                        max_depth=max_depth, 
                                        learning_rate=best_params['learning_rate'], 
                                        subsample=best_params['subsample'], 
                                        min_samples_split=best_params['min_samples_split'],  
                                        random_state=42)

        cvs = cross_val_score(gbr, X_train, y_train, cv=kfold, n_jobs=-1, scoring='r2').mean()
        CVS_matrix[i, j] = cvs
        
        gbr.fit(X_train, y_train)
        y_test_pred = gbr.predict(X_test)
        
        rmse_matrix[i, j] = np.sqrt(mean_squared_error(y_test, y_test_pred))
        R2_matrix[i, j] = r2_score(y_test, y_test_pred)

In [ ]:
# --- Performance Visualization (Heatmaps) ---
xticks = max_depth_range
yticks = n_estimators_range

plt.figure(figsize=(10, 6), dpi=600)
sns.heatmap(CVS_matrix, annot=True, xticklabels=xticks, yticklabels=yticks, cmap="YlGnBu")
plt.title('CVS Heatmap (n_estimators vs max_depth)')
plt.xlabel('max_depth')
plt.ylabel('n_estimators')
save_path = os.path.join(gbdt_path, 'CVS_heatmap.png')
plt.savefig(save_path, bbox_inches='tight')
plt.show()

In [ ]:
plt.figure(figsize=(10, 6), dpi=600)
sns.heatmap(rmse_matrix, annot=True, xticklabels=xticks, yticklabels=yticks, cmap="YlGnBu")
plt.title('RMSE Heatmap (n_estimators vs max_depth)')
plt.xlabel('max_depth')
plt.ylabel('n_estimators')
save_path = os.path.join(gbdt_path, 'RMSE_heatmap.png')
plt.savefig(save_path, bbox_inches='tight')
plt.show()

In [ ]:
plt.figure(figsize=(10, 6), dpi=600)
sns.heatmap(R2_matrix, annot=True, xticklabels=xticks, yticklabels=yticks, cmap="YlGnBu")
plt.title('R2 Score Heatmap (n_estimators vs max_depth)')
plt.xlabel('max_depth')
plt.ylabel('n_estimators')
save_path = os.path.join(gbdt_path, 'R2_heatmap.png')
plt.savefig(save_path, bbox_inches='tight')
plt.show()

In [ ]:
# --- Model Interpretation: Feature Importance ---
# n_estimators and max_depth need to be manually selected neither overfitting nor underfitting based on the three previously calculated Heatmaps
best_gbr = GradientBoostingRegressor(n_estimators=35, 
                                        max_depth=4, 
                                        learning_rate=0.2, 
                                        subsample=1.0, 
                                        min_samples_split=11,  
                                        random_state=42)

best_gbr.fit(X_train, y_train)

feature_importance = best_gbr.feature_importances_
features = X.columns

print('feature importance: {}'.format(best_gbr.feature_importances_))

plt.figure(figsize=(10, 6), dpi=600)
sns.barplot(x=feature_importance, y=features, palette=['#fcbba1', '#fff5f0', '#cb181d', '#fee0d2', '#fc9272', '#ef3b2c','#fb6a4a'])
plt.title('Feature Importance')
save_path = os.path.join(gbdt_path, 'Feature_Importance.png')
plt.savefig(save_path, bbox_inches='tight')
plt.show()

In [ ]:
# --- Model Interpretation: SHAP Analysis ---
explainer = shap.Explainer(best_gbr, X_train)
shap_values = explainer(X_test)
shap.summary_plot(shap_values, X_test, plot_type="bar")

shap_importance_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Mean SHAP Importance': np.abs(shap_values.values).mean(axis=0)
})

shap_values_df = pd.DataFrame(shap_values.values, columns=X_train.columns)
feature_values_df = pd.DataFrame(X_test.values, columns=X_train.columns)
detailed_shap_df = pd.concat([feature_values_df, shap_values_df, shap_importance_df], axis=1, keys=['Feature Value', 'SHAP Value','Mean SHAP Importance'])

print("Detailed SHAP DataFrame:")
print(detailed_shap_df.head())

In [ ]:
# --- Model Visualization: Tree Structure ---
feature_importance_matrix = np.array(feature_importance)
plt.figure(figsize=(35, 10), dpi=600)
plot_tree(best_gbr.estimators_[0, 0], filled=True, feature_names=features)
plt.title('Visualization of One Decision Tree from Gradient Boosting', fontsize=20 )
save_path = os.path.join(gbdt_path, 'Decision_Tree_Visualization_GBDT.png')
plt.savefig(save_path, bbox_inches='tight')
plt.show()

In [ ]:
# --- Export Results to Excel ---
cvs_matrix_df = pd.DataFrame(CVS_matrix, index=n_estimators_range, columns=max_depth_range)
rmse_matrix_df = pd.DataFrame(rmse_matrix, index=n_estimators_range, columns=max_depth_range)
r2_matrix_df = pd.DataFrame(R2_matrix, index=n_estimators_range, columns=max_depth_range)
feature_importance_matrix_df = pd.DataFrame(feature_importance_matrix, index=features)

excel_path = os.path.join(gbdt_path, 'GBDT_Output.xlsx')

with pd.ExcelWriter(excel_path) as writer:
    cvs_matrix_df.to_excel(writer, index=True, sheet_name='CVS_matrix')
    rmse_matrix_df.to_excel(writer, index=True, sheet_name='rmse_matrix')
    r2_matrix_df.to_excel(writer, index=True, sheet_name='R2_matrix')
    feature_importance_matrix_df.to_excel(writer, index=True, sheet_name='feature_importance_matrix', header=None)
    detailed_shap_df.to_excel(writer, index=True, sheet_name='shap_value')

In [ ]:
# --- Extract Sample Details from Individual Tree Nodes ---
first_tree = best_gbr.estimators_[0, 0]
node_indicator = first_tree.decision_path(X_train)
sample_data = X_train.copy()
sample_data['Target'] = y_train.values

output_file = os.path.join(gbdt_path, 'Tree_All_Nodes_Sample_Details.xlsx')

with pd.ExcelWriter(output_file) as writer:
    for node_id in range(first_tree.tree_.node_count):
        sample_index = node_indicator[:, node_id].toarray().ravel() == 1
        node_samples = sample_data[sample_index] 
        node_samples = node_samples.reset_index(drop=True)
        
        sheet_name = f"Node_{node_id}_Samples_{len(node_samples)}"
        node_samples.to_excel(writer, sheet_name=sheet_name, index=False)